In [1]:
from data_model import *
from decimal import Decimal
from pprint import pprint
from time import time
from copy import deepcopy
import pandas as pd # should not be used
pd.set_option('display.max_columns', 50)

import re, os, pickle
import itertools
import random, string
import collections
from utils import *

In [2]:
# BASE 104 MXG2000 A 34761934 TRADING MX00000000033560259
# reg_expr = r"104 RETTIL AANNKXPUNIN 2 A \d{8}"
# reg_expr = r"BASE 104 MXG2000 A \d{8} TRADING MX0{9}[1-9]\d{7}"
# reg_expr = r"RETAIL BANKING 2 A \d{8}"
# reg_expr = r"ACH A \d{8} \d{3}-\d{5}" # BAD
# reg_expr = r"ISLIG001\s\d{2}-[A-Z]{3}-\d{2}\s104\sIBG\sA\s\d{8}.*\d{22}[A-Z0-9]*"

In [14]:
##
## Export matched transactions 
## 

all_matched = arr_mtch2 + arr_mtch3 + arr_mtch4

output = []
for x in all_matched:
    set_id = x.set_id.split('_')[1]
    xs = x.item_id.split('_')
    item_id = xs[0]
    file_id = xs[1]
    file_name = file_dict[file_id].split('.')[0]
    output.append((file_name, set_id, item_id))
    
output = sorted(output, key = lambda x : x[0])
stats = {k:list(v) for k,v in itertools.groupby(output, key = lambda x : x[0])}

with open("matched_trns.pkl", 'wb') as f:
    pickle.dump(stats, f)

In [62]:
data = pd.DataFrame(arr_all, columns=['Set ID', 'Amount', 'Type', 'Match ID', 'Item ID', 'REF_KEY', 'SL_type']) 
PATH2 = "/Users/sasha/Work/Projects/matching_experiment/data/OCBC2_EX/"
data.to_excel(PATH2 + 'ALL.xlsx', index=False, header = True)

In [51]:
def match_key(arr_key):
    arr_key = sorted(arr_key, key=get_comb_key)
    groups = [list(value) for key, value in groupby(arr_key, key=get_comb_key)]
    tr_matched = []
    for group in groups:
        net_sum = sum(x.amount for x in group)
        if net_sum == 0:
            #print("Complete match found")
            tr_matched += [x.item_id for x in group]
    return tr_matched

In [77]:
len([x.ref_key for x in arr_all7 if re.search("MX0{9}[1-9]\d{8}", x.ref_key)])
# this needs additional investigation
arr_www = create_uk(arr_all7, r"MX0{9}[1-9]\d{7}-\d+-\d{8}", lambda x : x.split("-")[2])
tr_matched = match_key(arr_www)
tr_matched_set = set(tr_matched)
assert len(tr_matched_set) == len(tr_matched)
#print("Number of transactions covered:", len(arr_www))
#consist_un = check_consistency(arr_www, exclude_unique = True)
#consist_nn = check_consistency(arr_www, exclude_unique = False)
#print("Consistency without singles: ", consist_un)
#print("Consistency with singles: ", consist_nn)

NameError: name 'match_key' is not defined

In [18]:
arr_all8 = [x for x in arr_all7 if not x.item_id in tr_matched_set]

In [19]:
print(len(arr_all7))
print(len(arr_all8))

19083
19079


In [20]:
arr_mtch9, arr_all9 = unique_pair(arr_all8)
print(len(arr_mtch9))
print(len(arr_all9))

Correct: 0 0.0
Incorrect: 0 0.0
0
19079


In [21]:
#data = pd.DataFrame(arr_all7, columns=['Set ID', 'Amount', 'Type', 'Match ID', 'Item ID', 'REF_KEY', 'SL_type']) 
#PATH2 = "/Users/sasha/Work/Projects/matching_experiment/data/OCBC2_EX/"
#data.to_excel(PATH2 + 'ALL7.xlsx', index=False, header = True)

In [39]:
#def partial_match_homogenization(pms):
    # list of rartial matches
# pms = arr_partial
# pms = sorted(pms, key = lambda x: x.set_id)
# pms_groups = [list(v) for k,v in itertools.groupby(pms, key = lambda x: x.set_id)]

# pms = arr_partial
# pms = sorted(pms, key = lambda x: x.set_id)
# pms_groups = [list(v) for k,v in itertools.groupby(pms, key = lambda x: x.set_id)]

In [29]:
# arr_l = [x for x in arr_all if x.sl_type == 'L']
# arr_s = [x for x in arr_all if x.sl_type == 'S']

In [30]:
# xs_l = [x.ref_key.split()[0] for x in arr_l if not re.search(r"^\d{3}-\d{6}", x.ref_key)]

In [31]:
# import collections
# counter_l=collections.Counter(xs_l)
# counter_s=collections.Counter(xs_s)
# vals = list(counter.values())
# keys = list(counter.keys())
# xs = [keys[i] for i in range(len(vals)) if vals[i] == 1]

In [32]:
#xs_s

In [33]:
#counter_l.most_common()

In [34]:
# xs = [x for x in arr_all if re.search(r"^\S*\s\+\d+$",x.ref_key)]
# xs = [x for x in arr_all if re.search(r"^EQ00",x.ref_key)]
# len(xs)

In [15]:
## pattern for unique keys r"A \d{8} INW CLRG-\d{3}-"

In [63]:
arr_www = create_uk(arr_all4, r"A\s\d{8}.*\sTL", lambda x : x[2:10] + '-' + x[-25:(-3)])
print("Number of transactions covered:", len(arr_www))
consist_un = check_consistency(arr_www, exclude_unique = True)
#consist_nn = check_consistency(arr_www, exclude_unique = False)
print("Consistency without singles: ", consist_un)
#print("Consistency with singles: ", consist_nn)

Number of transactions covered: 2756
Number of complete matches: 48
Consistency without singles:  (1139, 48, 1187)


In [37]:
1 - 14689 / (17532 + 17531)

0.581068362661495

In [50]:
19079 - 1138 - 2153 - 579 - 520

14689

In [38]:
17532 + 17159

34691

In [61]:
print("Coverage", 1 - (19079 - (1138)) / len(arr_all))

Coverage 0.5300193495899206


In [44]:
# create partial matches
arr_key = arr_www
arr_key = sorted(arr_key, key=get_comb_key)
groups = [list(value) for key, value in groupby(arr_key, key=get_comb_key)]

In [45]:
partial_matches = []
complete_matches = []
for match in groups:
    set_id = match[0].set_id
    net_sum = sum(x.amount for x in match)
    if len(match) > 1:
        if net_sum == 0:
            complete_matches += match
        else:
            item_ids = [x.item_id for x in match]
            pm = PartialMatch(
                set_id=set_id,
                amount=net_sum,
                type_id=list(set([x.type_id for x in match])),
                match_id=None,
                item_id = "_".join(item_ids),
                item_ids=item_ids,
                ref_key=None,
                sl_type=list(set([x.sl_type for x in match])))
            partial_matches.append(pm)

In [46]:
partial_matches[0]

PartialMatch(set_id='ID0-105-219-21637-999-99-999-USD', amount=Decimal('-604000.2'), type_id=['DR'], match_id=None, item_id='400707398-0_400707399-0_400707446-0_400707350-0', item_ids=['400707398-0', '400707399-0', '400707446-0', '400707350-0'], ref_key=None, sl_type=['L'])

In [49]:
item_ids = []
for pm in partial_matches:
    item_ids += pm.item_ids

item_ids_set = set(item_ids)
print(len(item_ids_set))
print(len(item_ids))
#assert len(item_ids) == len(item_ids_set)

9608
9613


In [50]:
arr_all5 = [x for x in arr_all4 if not x.item_id in item_ids_set]

In [51]:
#item_ids_set

In [52]:
print(len(arr_all4))
print(len(arr_all5))

126342
116728


In [53]:
partial_matches[20]

PartialMatch(set_id='ID0-401-464-21693-999-99-999-SGD', amount=Decimal('-9437.70'), type_id=['DR'], match_id=None, item_id='400496469-0_400496464-0_400496463-0_400496461-0_400496462-0_400496459-0_400496454-0_400496455-0_400496456-0_400496457-0_400496458-0_400496471-0_400496472-0_400496473-0', item_ids=['400496469-0', '400496464-0', '400496463-0', '400496461-0', '400496462-0', '400496459-0', '400496454-0', '400496455-0', '400496456-0', '400496457-0', '400496458-0', '400496471-0', '400496472-0', '400496473-0'], ref_key=None, sl_type=['L'])

In [54]:
arr_all5_ = arr_all5 + partial_matches

In [55]:
#[x for x in arr_all5 if x.set_id == 'ID0-401-501-21886-999-99-999-SGD']

In [56]:
type(arr_all5_[111310])

utils.Transaction

In [57]:
print(len(arr_all5_))

119041


In [58]:
arr_mtch6, arr_all6 = unique_pair(arr_all5_)
print(len(arr_mtch6))
print(len(arr_all6))

Correct: 288 0.24
Incorrect: 172 0.14
632
118409


In [42]:
#arr_mtch6

In [25]:
def is_ref_consist(ref, arr):
    xs = [x for x in arr if re.search(ref, x.ref_key)]
    lnxs = len(xs)
    net_amount = sum([x.amount for x in xs])
    num_matches = len(set([x.match_id for x in xs]))
    
    if num_matches > 1:
        return (0, lnxs) # non-unique reference
    if num_matches == 1 and lnxs > 1 and net_amount != 0:
        return (1, lnxs)
    if num_matches == 1 and lnxs > 1 and net_amount == 0:
        return (2, lnxs)
    return (3, lnxs)

In [26]:
def get_candidates(xs, min_len = 3):
    lnx = len(xs)
    out = []
    for i in range(lnx):
        for j in range(i+min_len, lnx+1):
            candidate = xs[i:j]
            candidate = re.sub(r"^\s+", "", candidate)
            candidate = re.sub(r"\s+$", "", candidate)
            out.append(candidate)
    return set(out)

In [28]:
# this code checks what part of string is important, could be slow
splits = get_candidates("F9S1803126548600", min_len = 6)

t0 = time()
out = [(spl,) + is_ref_consist(spl, arr_all) for spl in splits]
refs = [x for x in out if x[1] in {1,2}]
t1 = time() - t0

print(t1)

12.246040105819702


In [35]:
refs_sorted = sorted(refs, key = lambda x: (x[1], x[2], 1/len(x[0])))
pprint(refs_sorted)
suggested_ref = refs_sorted[-1]
print("Suggested Ref: ", suggested_ref[0])

[('F9S1803126548600', 2, 6),
 ('F9S180312654860', 2, 6),
 ('9S1803126548600', 2, 6),
 ('9S180312654860', 2, 6),
 ('F9S18031265486', 2, 6),
 ('S1803126548600', 2, 6),
 ('F9S1803126548', 2, 6),
 ('1803126548600', 2, 6),
 ('S180312654860', 2, 6),
 ('9S18031265486', 2, 6),
 ('180312654860', 2, 6),
 ('803126548600', 2, 6),
 ('F9S180312654', 2, 6),
 ('S18031265486', 2, 6),
 ('9S1803126548', 2, 6),
 ('03126548600', 2, 6),
 ('9S180312654', 2, 6),
 ('18031265486', 2, 6),
 ('F9S18031265', 2, 6),
 ('80312654860', 2, 6),
 ('S1803126548', 2, 6),
 ('8031265486', 2, 6),
 ('0312654860', 2, 6),
 ('F9S1803126', 2, 6),
 ('1803126548', 2, 6),
 ('3126548600', 2, 6),
 ('S180312654', 2, 6),
 ('9S18031265', 2, 6),
 ('S18031265', 2, 6),
 ('803126548', 2, 6),
 ('126548600', 2, 6),
 ('312654860', 2, 6),
 ('180312654', 2, 6),
 ('F9S180312', 2, 6),
 ('9S1803126', 2, 6),
 ('031265486', 2, 6),
 ('31265486', 2, 6),
 ('18031265', 2, 6),
 ('F9S18031', 2, 6),
 ('80312654', 2, 6),
 ('26548600', 2, 6),
 ('S1803126', 2, 6)

In [14]:
# this needs additional investigation
# xs = [x.ref_key for x in arr_all if re.search(r"\sTLGL", x.ref_key)]
# xs = [x.ref_key for x in arr_all if re.search(r"A\s\d{8}.*\sTLGL", x.ref_key)]
# print(len(xs))

In [78]:
arr_www = create_uk(arr_all9, r"MX00[1-9]\d{11}", lambda x : x[4:12]) + create_uk(arr_all9, r"MX0{9}[1-9]\d{7}", lambda x : x[-8:])

print("Number of transactions covered:", len(arr_www))
consist_un = check_consistency(arr_www, exclude_unique = True)
consist_nn = check_consistency(arr_www, exclude_unique = False)
print("Consistency without singles: ", consist_un)
print("Consistency with singles: ", consist_nn)

TypeError: expected string or bytes-like object

In [20]:
uks = set([x.unique_key for x in arr_www]) 

In [ ]:
# the above transactions could be matched with non-pattern transactions

In [33]:
# here is an example for a single key
uk = '33900287'
tmp = [x for x in arr_all if re.search(uk, x.ref_key)]
tmp = sorted(tmp, key = lambda x : x.set_id)
groups = [list(value) for key, value in itertools.groupby(tmp, key=lambda x: x.set_id)]
for group in groups:
    len_set = len(set([x.match_id for x in group]))
    amount = sum([x.amount for x in group])
    pprint((len(group), len_set, amount))

(1, 1, Decimal('-492180'))
(1, 1, Decimal('-492180'))


In [34]:
# example for all the keys, slow
out = []
for uk in uks:
    tmp = [x for x in arr_all if re.search(uk, x.ref_key)]
    tmp = sorted(tmp, key = lambda x : x.set_id)
    groups = [list(value) for key, value in itertools.groupby(tmp, key=lambda x: x.set_id)]
    for group in groups:
        len_set = len(set([x.match_id for x in group]))
        amount = sum([x.amount for x in group])
        out.append((len(group), len_set, amount))

In [38]:
len1 = sum(x[0] for x in out if x[1] == 1)
len2 = sum(x[0] for x in out if x[1] != 1)